In [ ]:
pip install selenium

# Bibliotecas

In [ ]:
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# Criar Uma Instância do Chrome

In [ ]:
service = Service()

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)

# Coletar Dados

Navegar até o site do DATASUS

In [ ]:
url = 'http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sih/cnv/spabr.def'

driver.get(url)

Coletar Linhas da Consulta Gerada na Nova Aba e Concatená-las

In [ ]:
lin = ""
arqs = []
colunas = ['Município', 
           '01 Ações de promoção e prevenção em saúde',
           '02 Procedimentos com finalidade diagnóstica',
           '03 Procedimentos clínicos',
           '04 Procedimentos cirúrgicos',
           '05 Transplantes de orgãos, tecidos e células',
           '06 Medicamentos',
           '07 Órteses, próteses e materiais especiais',
           '08 Ações complementares da atenção à saúde',
           'QTD TOTAL',
           'vazio']


# Em "Coluna" Selecionar a opção "Grupo Procedimento"
driver.find_element(By.XPATH, '//*[@id="C"]/option[7]').click()

# Deselecionar o Periodo
driver.find_element(By.XPATH, f'//*[@id="A"]/option[1]').click()

for periodo in range(192):   
    arq = []
    
    # Selecionar Periodo
    driver.find_element(By.XPATH, f'//*[@id="A"]/option[{periodo+1}]').click()
    data = driver.find_element(By.XPATH, f'//*[@id="A"]/option[{periodo+1}]').text + ','
    
    # Clicar em Mostrar
    driver.find_element(By.XPATH, '/html/body/div/div/center/div/form/div[4]/div[2]/div[2]/input[1]').click()
    
    # Ir até nova aba
    janelas_abertas = driver.window_handles
    driver.switch_to.window(janelas_abertas[1])
    
    # Ler os registros em Uma Lista de Listas
    for row in range(2379):
        for col in range(10):
            lin += driver.find_element(By.XPATH, f'/html/body/div/div/table[1]/tbody/tr[{row+1}]/td[{col+1}]').text + ',' 
        arq.append(lin.split(','))  
        lin = ""     
    
    # Fechar Aba atual e navegar até primeira aba
    driver.close()
    janelas_abertas = driver.window_handles
    driver.switch_to.window(janelas_abertas[0])    
    
    # Deselecionar Periodo
    driver.find_element(By.XPATH, f'//*[@id="A"]/option[{periodo+1}]').click()
    
    # Ler e guardar dataframe com outros periodos
    df = pd.DataFrame(arq, columns=colunas)
    df['Data'] = data
    arqs.append(df)
    
#Concatenar arquivos
df_resultado = pd.concat(arqs, ignore_index=True)

# Exportar Arquivo
df_resultado.to_csv('procedimentos_datasus_16_anos.csv', index=False)